In [1]:
!pip install catboost xgboost

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
data_df = pd.read_csv("/kaggle/input/imdb-final/data_selected_2.csv")
data_df.drop_duplicates(inplace=True)
X = data_df.drop('Rating', axis=1)
Y = data_df['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [5]:
print("Training shape (X, Y):", X_train.shape, y_train.shape)
print("Test shape (X, Y):", X_test.shape, y_test.shape)

Training shape (X, Y): (17718, 16) (17718,)
Test shape (X, Y): (7594, 16) (7594,)


In [6]:
def get_accuracy_details(model, X,y):
    """
    Calculates and prints accuracy details for a given model using provided features and target data.

    Args:
        model: A trained machine learning model with `predict` and `score` methods.
        X: Features data used for making predictions.
        y: True target values corresponding to the features.

    Prints:
        Validation R² score: Coefficient of determination of the prediction.
        Validation Mean Squared Error: Mean of the squared differences between predicted and true values.
        Validation Mean Absolute Error: Mean of the absolute differences between predicted and true values.
    """

    y_pred = model.predict(X)

    # Calculate R² score
    r2_score = model.score(X, y)

    from sklearn.metrics import mean_squared_error, mean_absolute_error
    val_mse = mean_squared_error(y, y_pred)
    val_mae = mean_absolute_error(y, y_pred)

    print(f"Validation R² score: {r2_score}")
    print(f"Validation Mean Squared Error: {val_mse}")
    print(f"Validation Mean Absolute Error: {val_mae}")

In [7]:
from sklearn.model_selection import GridSearchCV

In [8]:
from sklearn.ensemble import RandomForestRegressor

def train_random_forest(X_train, y_train):
    param_grid = {
        'n_estimators': [500, 800],
        'max_depth': [10, 15],
    }
    rf_regressor = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(rf_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_rf_model = grid_search.best_estimator_

    return best_rf_model, best_params

best_rf_model, best_params_rf = train_random_forest(X_train, y_train)
print(f"Random Forest - Best Parameters: {best_params_rf}")
get_accuracy_details(best_rf_model, X_test, y_test)

Random Forest - Best Parameters: {'max_depth': 15, 'n_estimators': 800}
Validation R² score: 0.7786886336207179
Validation Mean Squared Error: 0.2919767093812067
Validation Mean Absolute Error: 0.3825317247577738


In [9]:
from sklearn.neural_network import MLPRegressor

def train_mlp(X_train, y_train):
    param_grid = {
        'hidden_layer_sizes': [(32, 256, 1024, 256, 32), (32, 256, 1024, 1024, 256, 32)],
        'activation': ['relu'],
        'solver': ['adam'],
        'alpha': [0.01],
        'learning_rate_init': [0.1]
    }
    mlp_regressor = MLPRegressor(random_state=42, max_iter=200, early_stopping=True, batch_size=1024)
    grid_search = GridSearchCV(mlp_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_mlp_model = grid_search.best_estimator_

    return best_mlp_model, best_params

best_mlp_model, best_params_mlp = train_mlp(X_train, y_train)
print(f"MLP - Best Parameters: {best_params_mlp}")

print("MLP Model Performance on Test Set:")
get_accuracy_details(best_mlp_model, X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP - Best Parameters: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (64, 64), 'learning_rate_init': 0.1, 'solver': 'adam'}
MLP Model Performance on Test Set:
Validation R² score: -332139953.77383536
Validation Mean Squared Error: 438193178.39594567
Validation Mean Absolute Error: 3930.9658100534716


In [10]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0


In [16]:
import xgboost as xgb

def train_xgboost(X_train, y_train):
    param_grid = {
        'n_estimators': [500, 800, 1200],
        'learning_rate': [0.1],
        'max_depth': [15, 10, 20],
    }
    xgb_regressor = xgb.XGBRegressor(device="cuda", random_state=42, objective='reg:squarederror')
    grid_search = GridSearchCV(xgb_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_xgb_model = grid_search.best_estimator_

    return best_xgb_model, best_params

best_xgb_model, best_params_xgb = train_xgboost(X_train, y_train)
print(f"XGBoost - Best Parameters: {best_params_xgb}")

print("XGBoost Model Performance on Test Set:")
get_accuracy_details(best_xgb_model, X_test, y_test)


d:\Python\Columbina\Lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The XGBRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [18]:
import xgboost as xgb
import sklearn
print(f"XGBoost version: {xgb.__version__}")
print(f"Scikit-Learn version: {sklearn.__version__}")


XGBoost version: 2.1.3
Scikit-Learn version: 1.6.0


In [21]:
from catboost import CatBoostRegressor

def train_catboost(X_train, y_train):
    param_grid = {
        'iterations': [500, 800, 1200],
        'learning_rate': [0.1],
        'depth': [10, 15, 20]
    }
    cb_regressor = CatBoostRegressor(random_seed=29, verbose=0, task_type="GPU", devices='0')
    grid_search = GridSearchCV(cb_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_cb_model = grid_search.best_estimator_
    
    return best_cb_model, best_params

best_cb_model, best_params_cb = train_catboost(X_train, y_train)
print(f"CatBoost - Best Parameters: {best_params_cb}")
get_accuracy_details(best_cb_model, X_test, y_test)

: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Collect the performance metrics
metrics = {}

def collect_metrics(model, name, X_test, y_test):
    y_pred = model.predict(X_test)
    r2 = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    metrics[name] = {'R2': r2, 'MSE': mse, 'MAE': mae}

collect_metrics(best_cb_model, 'CatBoost', X_test, y_test)
collect_metrics(best_rf_model, 'Random Forest', X_test, y_test)
collect_metrics(best_mlp_model, 'MLP', X_test, y_test)
collect_metrics(best_xgb_model, 'XGBoost', X_test, y_test)

# Visualize the metrics (Improved with color palette and log scale for MSE)
metric_names = ['R2', 'MSE', 'MAE']
model_names = list(metrics.keys())

# --- Create the plot ---
fig, ax = plt.subplots(figsize=(12, 6))

# --- Bar width and positions ---
bar_width = 0.2
x = np.arange(len(metric_names))

# --- Color Palette ---
colors = plt.cm.Set2(np.linspace(0, 1, len(model_names)))  # Using a colormap

# --- Plotting the bars for each model ---
for i, model_name in enumerate(model_names):
    metric_values = [metrics[model_name][metric] for metric in metric_names]
    rects = ax.bar(x + i * bar_width, metric_values, bar_width, label=model_name, color=colors[i])
    ax.bar_label(rects, padding=3, fmt='%.3f')

# --- Customize the plot ---
ax.set_xlabel('Metrics', fontweight='bold')
ax.set_ylabel('Value', fontweight='bold')
ax.set_title('Model Comparison - Performance Metrics', fontweight='bold', fontsize=14)
ax.set_xticks(x + bar_width * (len(model_names) - 1) / 2)
ax.set_xticklabels(metric_names)
ax.legend()

# --- Log Scale for MSE ---
ax.set_yscale('log')

# --- Adjust layout and display ---
fig.tight_layout()
plt.show()

NameError: name 'best_cb_model' is not defined

In [12]:
# --- Save the model (JSON format) ---
best_xgb_model.save_model("xgboost_model.json")
best_cb_model.save_model("catboost_model.cbm")

In [ ]:
# --- Save the model (JSON format) ---
best_cb_model.save_model("catboost_model.json")